In [ ]:
# !pip install -e ..

In [ ]:
# !pip install notion-client

In [ ]:
import collections
import datetime
import pathlib
import tqdm
import aiohttp
from notion_client import AsyncClient
import re
import json

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from assistance.api.keys import load_secret

In [ ]:
API_KEY = load_secret("reporting_notion_api_key")
DATABASE_ID = "8e62c1f5c4d545c3a529fb0bc549ad65"

In [ ]:
notion = AsyncClient(auth=API_KEY)

In [ ]:
response = await notion.databases.retrieve(database_id=DATABASE_ID)

In [ ]:
response['properties']

In [ ]:
# PROMPT = 'I%5E%7D%7C'
TRANSCRIPT_START = '%40%5Bw%5C'
TRANSCRIPT_OVERFLOW_1 = 'skWG'
TRANSCRIPT_OVERFLOW_2 = 'i%7D%60V'
TRANSCRIPT_OVERFLOW_3 = 'ovXV'
TRANSCRIPT_OVERFLOW_4 = 'N%3AUg'
FORM = 'WZEz'
EMAIL = 'Wk%3Dt'
NAME = 'title'
DATE = '%60jg_'
USER_ID = 'brjj'

In [ ]:
transcripts_dir = pathlib.Path.home() / ".assistance.chat" / "transcripts"

In [ ]:
paths = list(transcripts_dir.glob('*/*'))

In [ ]:
collected_transcripts = collections.defaultdict(list)

In [ ]:
for path in paths:
    timestamp = datetime.datetime.strptime(path.parts[-1], '%Y%m%dT%H%M%S_%fZ')
    user_id = path.parts[-2]
    
    collected_transcripts[user_id].append(timestamp)

sorted_collected_transcripts = {}
for user_id, timestamps in collected_transcripts.items():
    sorted_collected_transcripts[user_id] = sorted(timestamps)
    

latest_transcript = {}
for user_id, timestamps in sorted_collected_transcripts.items():
    latest_transcript[user_id] = timestamps[-1]
    
paths_to_use = {}
for user_id, timestamp in latest_transcript.items():
    path = transcripts_dir / user_id / datetime.datetime.strftime(timestamp, '%Y%m%dT%H%M%S_000Z')
    assert path.exists()
    
    paths_to_use[user_id] = path

In [ ]:
rows = []
start_cursor = None

while True:
    response = await notion.databases.query(database_id=DATABASE_ID, start_cursor=start_cursor)
    
    rows += response['results']
    if not response['has_more']:
        break
        
    else:
        start_cursor = response['next_cursor']

In [ ]:
# notion.databases.query?

In [ ]:
# response['results']

In [ ]:
list_of_user_ids = [
    row['properties']['User ID']['rich_text'][0]['text']['content'] 
    for row in rows
]

user_ids = set(list_of_user_ids)

assert len(user_ids) == len(list_of_user_ids)

In [ ]:
len(list_of_user_ids)

In [ ]:
response.keys()

In [ ]:
for user_id in user_ids:
    if list_of_user_ids.count(user_id) == 1:
        continue
        
    row_ids = [row["id"] for row in rows if row['properties']['User ID']['rich_text'][0]['text']['content'] == user_id]
    
    for row_id in row_ids[1:]:
        await notion.blocks.delete(row_id)

In [ ]:
# for row in rows:
#     await notion.blocks.delete(row["id"])

In [ ]:
regex_pattern = r'\{(?:[^{}]|())*\}'

In [ ]:
for user_id, path in tqdm.tqdm(paths_to_use.items()):
    if user_id in user_ids:
        continue
    
    timestamp = datetime.datetime.strptime(path.parts[-1], '%Y%m%dT%H%M%S_%fZ')
    timestamp = datetime.datetime.strftime(timestamp, '%Y-%m-%dT%H:%M:%S.000Z')


    with open(path) as f:
        transcript = f.read()

    try:
        # Pulling the form out of the transcript isn't ideal. Should be saved separately.
        form = re.search(regex_pattern, transcript).group(0)
        
        details = json.loads(form)
        name = details["Name"]
        email = details["Email"]
    except:
        continue
    
    # prompt = transcript.split("Go!\n\n")[0] + "Go!"
    
    transcript_user = transcript.split("Go!\n\n")[-1]
    
    split = "\n\n"
    line_by_line = transcript_user.split(split)
    line_by_line = [item for item in line_by_line if len(item) != 0]
    
    # This is a nasty hack with messy code for notion limiting inputs to 2000 characters.
    # This will still break if above 8000.
    if len(transcript_user) > 8000:
        part_a = len(line_by_line)//5
        part_b = part_a * 2
        part_c = part_a * 3
        part_d = part_a * 4
        
        transcript_start = split.join(line_by_line[0:part_a])
        transcript_overflow_1 = split.join(line_by_line[part_a:part_b])
        transcript_overflow_2 = split.join(line_by_line[part_b:part_c])
        transcript_overflow_3 = split.join(line_by_line[part_c:part_d])
        transcript_overflow_4 = split.join(line_by_line[part_d::])
    
    elif len(transcript_user) > 6000:
        part_a = len(line_by_line)//4
        part_b = part_a * 2
        part_c = part_a * 3
        
        transcript_start = split.join(line_by_line[0:part_a])
        transcript_overflow_1 = split.join(line_by_line[part_a:part_b])
        transcript_overflow_2 = split.join(line_by_line[part_b:part_c])
        transcript_overflow_3 = split.join(line_by_line[part_c::])
        transcript_overflow_4 = ""
    
    elif len(transcript_user) > 4000:
        part_a = len(line_by_line)//3
        part_b = part_a * 2
        
        transcript_start = split.join(line_by_line[0:part_a])
        transcript_overflow_1 = split.join(line_by_line[part_a:part_b])
        transcript_overflow_2 = split.join(line_by_line[part_b::])
        transcript_overflow_3 = ""
        transcript_overflow_4 = ""
        
    elif len(transcript_user) > 2000:
        middle = len(line_by_line)//2

        transcript_start = split.join(line_by_line[0:middle])
        transcript_overflow_1 = split.join(line_by_line[middle::])
        transcript_overflow_2 = ""
        transcript_overflow_3 = ""
        transcript_overflow_4 = ""
        
    else:
        transcript_start = transcript_user
        transcript_overflow_1 = ""
        transcript_overflow_2 = ""
        transcript_overflow_3 = ""
        transcript_overflow_4 = ""
    
    data = {
        # PROMPT:  [{"type": "text", "text": {"content": prompt}}],
        TRANSCRIPT_START:  [{"type": "text", "text": {"content": transcript_start}}],
        TRANSCRIPT_OVERFLOW_1:  [{"type": "text", "text": {"content": transcript_overflow_1}}],
        TRANSCRIPT_OVERFLOW_2:  [{"type": "text", "text": {"content": transcript_overflow_2}}],
        TRANSCRIPT_OVERFLOW_3:  [{"type": "text", "text": {"content": transcript_overflow_3}}],
        TRANSCRIPT_OVERFLOW_4:  [{"type": "text", "text": {"content": transcript_overflow_4}}],
        FORM: [{"type": "text", "text": {"content": form}}],
        EMAIL: email,
        NAME: [{"type": "text", "text": {"content": name}}],
        DATE: {"start": timestamp},
        USER_ID: [{"type": "text", "text": {"content": user_id}}]
    }
    
    try:
        response = await notion.pages.create(parent={"database_id": DATABASE_ID}, properties=data)
    except:
        # Another nasty hack, just to get some useful results into the table
        pass

        

In [ ]:
len(transcript)

In [ ]:
data

In [ ]:
transcript.split("Go!\n\n")[-1]

In [ ]:
FORM

In [ ]:
with open(list(paths_to_use.values())[0], 'rb') as f:
    bytes_file_data = f.read()

In [ ]:
response = notion.

In [ ]:
path = paths[0]

In [ ]:
list_users_response = await notion.users.list()
list_users_response

In [ ]:
data = {

}

In [ ]:
async with aiohttp.ClientSession() as session:
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}"

    headers = {
        "accept": "application/json",
        "Notion-Version": "2022-06-28",
        "Authorization": f"Bearer {API_KEY}",
    }

    async with session.get(url, headers=headers) as resp:
        json = await resp.json()


In [ ]:
json

In [ ]:
async def get_all_sub_blocks(block_id):
    parameters = ""
    has_more = True
    results = []
    
    async with aiohttp.ClientSession() as session:
        while True:
            url = f"https://api.notion.com/v1/blocks/{block_id}/children{parameters}"

            headers = {
                "accept": "application/json",
                "Notion-Version": "2022-06-28",
                "Authorization": f"Bearer {API_KEY}",
            }

            async with session.get(url, headers=headers) as resp:
                try:
                    json = await resp.json()
                except ContentTypeError:
                    print(await resp.headers())
                    raise

            results += json['results']
            has_more = json['has_more']

            if not has_more:
                return results

            parameters = f"?start_cursor={json['next_cursor']}"

In [ ]:
async def get_transcript(block_id):
    a_page = await get_all_sub_blocks(block_id)
    
    transcript_items = []

    for item in a_page:
        for rich_text in item['paragraph']['rich_text']:
            transcript_items.append(rich_text['plain_text'])

    transcript = "\n\n".join(transcript_items)
    
    return transcript

In [ ]:
results = await get_all_sub_blocks(PARENT_PAGE_ID)

In [ ]:
# results

In [ ]:
pages = {
    item['id']: {'title': item['child_page']['title'], 'created_time': item['created_time']}
    for item in results
    if item['type'] == 'child_page'
}

In [ ]:
transcripts_dir = pathlib.Path.home() / ".assistance.chat" / "transcripts"

In [ ]:
for page_id, details in tqdm.tqdm(pages.items()):
    user_id = details['title']
    timestamp = details['created_time']
    
    user_dir = transcripts_dir / user_id
    user_dir.mkdir(exist_ok=True, parents=True)
    
    filename = timestamp.replace("-", "").replace(":", "").replace(".", "_")
    filepath = user_dir / filename
    
    if filepath.exists():
        continue
    
    transcript = await get_transcript(page_id)
    
    with open(filepath, "w") as f:
        f.write(transcript)